<a href="https://colab.research.google.com/github/skdldbwls/predict_mbti_korean/blob/master/mbti_prediction_with_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **준비 사항**

In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 1.5MB 12.9MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
     |████████████████████████████████| 2.9MB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=08c6d7f8364a286d0b172e963850b295bd47e7451076e77acda743948e9bf2e7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
from google.colab import files
myfile = files.upload()

Saving data_winter_final.csv to data_winter_final.csv


In [ ]:
data = pd.read_csv("data_winter_final.csv")
data.head()

,type,posts
0,ESTP,1. 최근 카페의 질의응답 게시판에 지속적으로 등업을 요구하는 글이 올라오고 있습니...
1,INFP,"제게 큰 현타를 주었던 글이에요.제 자신이 마음에 안 들고,늘 부족한 것 같고,절대..."
2,ISFJ,"유고슬라비아는 남슬라브 국가들(세르비아, 슬로베니아, 크로아티아, 보스니아, 북마케..."
3,ISTP,10대 여자고 키도 크고 나이에 비해 성숙한 이미지를 가지고 있어요. 잇팁답게 차갑...
4,ISFP,제가 요즘 끌리는 남자가있는데 esfp인거같아요esfj인줄 알았는데 상황에따라 행동...


## mbti라벨링

In [ ]:
# Split mbti personality into 4 letters and binarize
titles = ["Extraversion (E) - Introversion (I)",
          "Sensation (S) - INtuition (N)",
          "Thinking (T) - Feeling (F)",
          "Judgement (J) - Perception (P)"
         ] 
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]


#transform mbti to binary vector
def translate_personality(personality):
    return [b_Pers[l] for l in personality]

#transform binary vector to mbti personality
def translate_back(personality):
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s

list_personality_bin = np.array([translate_personality(p) for p in data.type])
print("Binarize MBTI list: \n%s" % list_personality_bin)

Binarize MBTI list: 
[[1 1 1 1]
 [0 0 0 1]
 [0 1 0 0]
 ...
 [0 0 0 0]
 [0 1 1 0]
 [0 1 0 1]]


In [ ]:

data['I-E'] = list_personality_bin[:,0]
data['N-S'] = list_personality_bin[:,1]
data['F-T'] = list_personality_bin[:,2]
data['J-P'] = list_personality_bin[:,3]


## train, test split

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data[['type']]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

strat_train_set = strat_train_set.reset_index(drop=True)
strat_test_set = strat_test_set.reset_index(drop=True)

X_train = strat_train_set.posts
X_test = strat_test_set.posts

print(X_train.shape)
print(X_test.shape)

(24910,)
(6228,)


In [ ]:
X_train

0        하나 바라는 점이 생겼습니다그것은 바로 성숙하고 의연하게 살아갈수 있는 인프피가 되...
1        관계성 + 주관적 생각, 느낌.현실에서 만난 사람들 대상.NT NF SJ SP순.E...
2        인프피로 바뀌었네요요새 외출을 못한 탓인가봐요과거의 저를 만났더라면 서로 비유법으로...
3        ​설명이 인프피 같지는 않네요?! 제 설명이 tj느낌..?다른 인프피분들은 건축학과...
4                                            ​​​​야근 당첨​​​​
                               ...                        
24905    저랑 제 연인이 둘 다 isfj 에요.사귄지는 이제 280일 정도 됐어요!​서로 잘...
24906    어떤 사랑을 하고 계신가요?가슴 뛰고 설레는 사랑?편안하고 아낌받는 사랑?​난 아직...
24907                                                  NaN
24908    ENTJ(사촌1, ENFP동생, 성인)😎ENFP(사촌2, ENTJ누나, 성인)😈IN...
24909    ENFP.... 새삼 느끼지만 그들에게선 부드러운 카리스마가 느껴집니다.뭐랄까, 사...
Name: posts, Length: 24910, dtype: object

In [ ]:
y_train = data['I-E'][train_index].values
y_test = data['I-E'][test_index].values

## 전처리

In [ ]:
sentences = strat_train_set['posts']
sentences[:10]

0    하나 바라는 점이 생겼습니다그것은 바로 성숙하고 의연하게 살아갈수 있는 인프피가 되...
1    관계성 + 주관적 생각, 느낌.현실에서 만난 사람들 대상.NT NF SJ SP순.E...
2    인프피로 바뀌었네요요새 외출을 못한 탓인가봐요과거의 저를 만났더라면 서로 비유법으로...
3    ​설명이 인프피 같지는 않네요?! 제 설명이 tj느낌..?다른 인프피분들은 건축학과...
4                                        ​​​​야근 당첨​​​​
5    1학년 때 저 혼자서 무지 친하다고 생각했던 친구가 둘 있었는데,(동아리...)제가...
6    현실에서 피자 먹으면서 얘기 나누고 싶은데 코로나는 언제 잠잠해질까요...ㅠ​온라인...
7                         눈인증이 유행인가요?저도 같이 눈도장 찍고 갑니다❤
8    INTJ : 귀찮INTP : 귀찮INFP : 귀찮INFJ : 귀찮ENTJ : 귀찮E...
9    갑자기 생각나서 풀어 보는 경험담입니다.​결혼 후 3년 정도 지나고 아내의 친구들과...
Name: posts, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 하나 바라는 점이 생겼습니다그것은 바로 성숙하고 의연하게 살아갈수 있는 인프피가 되고 싶다..모든게 날것 그대로 였던 20대때는 참 많이 기쁘기도 행복하기도 슬프기도 절망스러웠기도 했던 나날의 연속이였어요내가 어디에 발을 내딛고 있는지어디에 중심을 서고 있는지두려움과 소용돌이 치는 감정 안에서누구나 봐도 항상 불안정하고 위태로워 보였죠예민한 감정과 소용돌이 치는 마음들이힘들었지만 한편으론 나는 특별하다는 나르시즘에 갖혀 평범하게 살아가는 사람들의 가치를 깨닫지 못했어요우울증과 세상에 대한 두려움과 기피로 가득했던 사회생활을 전혀 할수 없을것만 같았던 저는어느새 INFP라는 프레임의 껍질에서 나와서 더러 사람들과 같은 평범하게 일을 하며 사는 길을 택했습니다감정에 덜 예민해지는게 나를 잃는것은 아닌지평범하게 일하면서 지내는게 내 영혼을 잠재우는게 아닌지걱정도 됐지만여리고 보호받아야만 하는 자아 이미지 상에서 벗어나감정에 의연해지고 성숙해 지고 싶었습니다우연인지 몰라도테스트 결과도 전이였으면 보호받아야 하는 여린 이미지가 나왔을 법도 한데지금은 좀더 강해진 모습이 투영된 모습이 나왔네요..자리가 사람을 만든다고전혀 변할수 없을거 같던 저도나이가 들고 사회적으로 아랫사람도 거느려야 하는 입장이 되니눈물많고 여리던 인프피의 모습에서만 머무르는것도 아니더군요..전에는 날 것 그대로의 감정 풍부한 저의 모습만이 제 모습이라고 믿고 살았지만살면서 다듬어지고 의연해지는게 인프피인 저에겐 예상치 못한 변화가 되었습니다마음적으로 힘들어하는 인프피들 아마 많을거에요..이세상이 인프피로서 적응하기가 그렇게 보드랍지가 못하니까요그렇다고 해서 예민하고 우울하고 슬픈 모습만이 전부이진 않을테니 시간이 지나면서 성숙해지고 다듬어 진다고 믿어보는것도 나쁘지 않을거 같아요정말 오랫만에 이런 글을 쓰는데..날것 그대로의 인프피도 매력 충만하지만 ^^;마음에 상처에 잠식 당하지 않는 성숙한 인프피의 모습이 되고 싶어요 [SEP]',
 '[CLS] 관계성 + 주관적 생각, 느낌.현실

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] 하나 바라는 점이 생겼습니다그것은 바로 성숙하고 의연하게 살아갈수 있는 인프피가 되고 싶다..모든게 날것 그대로 였던 20대때는 참 많이 기쁘기도 행복하기도 슬프기도 절망스러웠기도 했던 나날의 연속이였어요내가 어디에 발을 내딛고 있는지어디에 중심을 서고 있는지두려움과 소용돌이 치는 감정 안에서누구나 봐도 항상 불안정하고 위태로워 보였죠예민한 감정과 소용돌이 치는 마음들이힘들었지만 한편으론 나는 특별하다는 나르시즘에 갖혀 평범하게 살아가는 사람들의 가치를 깨닫지 못했어요우울증과 세상에 대한 두려움과 기피로 가득했던 사회생활을 전혀 할수 없을것만 같았던 저는어느새 INFP라는 프레임의 껍질에서 나와서 더러 사람들과 같은 평범하게 일을 하며 사는 길을 택했습니다감정에 덜 예민해지는게 나를 잃는것은 아닌지평범하게 일하면서 지내는게 내 영혼을 잠재우는게 아닌지걱정도 됐지만여리고 보호받아야만 하는 자아 이미지 상에서 벗어나감정에 의연해지고 성숙해 지고 싶었습니다우연인지 몰라도테스트 결과도 전이였으면 보호받아야 하는 여린 이미지가 나왔을 법도 한데지금은 좀더 강해진 모습이 투영된 모습이 나왔네요..자리가 사람을 만든다고전혀 변할수 없을거 같던 저도나이가 들고 사회적으로 아랫사람도 거느려야 하는 입장이 되니눈물많고 여리던 인프피의 모습에서만 머무르는것도 아니더군요..전에는 날 것 그대로의 감정 풍부한 저의 모습만이 제 모습이라고 믿고 살았지만살면서 다듬어지고 의연해지는게 인프피인 저에겐 예상치 못한 변화가 되었습니다마음적으로 힘들어하는 인프피들 아마 많을거에요..이세상이 인프피로서 적응하기가 그렇게 보드랍지가 못하니까요그렇다고 해서 예민하고 우울하고 슬픈 모습만이 전부이진 않을테니 시간이 지나면서 성숙해지고 다듬어 진다고 믿어보는것도 나쁘지 않을거 같아요정말 오랫만에 이런 글을 쓰는데..날것 그대로의 인프피도 매력 충만하지만 ^^;마음에 상처에 잠식 당하지 않는 성숙한 인프피의 모습이 되고 싶어요 [SEP]
['[CLS]', '하', '##나', '바', '##

In [ ]:
# 라벨 추출
labels = strat_train_set['I-E'].values
labels

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9952,  16439,   9318,  60362,   9668,  10739,   9420,
       118637, 119081,  48345,  78136, 118627,  10892,  71433,   9434,
       119063,  12453,   9637,  25486,  17594,   9408,  16985, 101202,
        15891,  13767,   9640,  28396,  97146,  11287,  66674,   9495,
        11903,    119,    119,  25701,  14153,   8985, 118627, 110589,
         9573,  23990,  10197,  14423, 118832,  11018,   9735,  47058,
         8932, 119022,  27792,   9966,  70915,  68100,   9479,  28396,
        27792,   9666,  89292,  12605,  30873, 119172,  27792,   9965,
        23990,   8982,  41919,  10459, 100208,  10739, 119147,  12965,
        48549,  31605,  11287,   9546,  48446,  10530,   9323,  10622,
          100,  13767,  12508,  12965,  48446,  10530,   9694,  86904,
         9425,  11664,  13767,  12508, 118802,  26737, 119169,  11882,
         9448,  24974, 118794,  10739,   9779,  11018,   8848,  16605,
         9521,  11489, 118751,  17196,  16439,   9363,  12092,   9959,
      

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   8924, 118729,   9069, 119276,  10739,  17342,  12092,   9551,
         37819,   9583,  12692,  13764,    100,   8982,  10892,    100,   9583,
         62211,   9495,  12965,  12424,  11903,  14040,   9574,  14871,   9924,
         38696,  14523,  48549,    119,    119,    119,   9330,  43962,  10622,
          9041,  12692,  14153,   9965,  54141,  25503,   9074,   9356,  12310,
          9685,  11664,  10017,  27023,  67527,  10892,   9077,  35506,  77884,
         48549,    119,   9011,  10739,  41605,   9095,  30858,  14871,   9944,
         56645,  86732,  21069,    131,  11025,  36210,  24017, 100699,    117,
          9638,   9095,  30858,  70969,   8934,  31728,    121,  57030,   9645,
         48345,    119,   9358,   9574,  14871,  15303,   8845, 119205, 118649,
         11664,  11287,   9928,  48533,  16855,   9647, 119081,  48345,    119,
         31191,  10622,   9357,  12945,  14523,  16323,  24982,  48549,    119,
           102,      0,      0,      0, 

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 전처리 - 테스트셋

In [ ]:
# 리뷰 문장 추출
sentences = strat_test_set['posts']
sentences[:10]

0    ​어제 저녁 8-9시쯤에 졸려서 잤는데 새벽 6시 50분에 일어났다가, 다시 자니 ...
1    그정도까진 아닌데 참 극단적으로 나오네요 ㅋㅋ 그래도 저번에 CCCCC 나온 것 보...
2    Entj여러분, 반복된 같은 행동 잘 질려하시나요?반복해서 꾸준히 하는 게 좋다는데...
3    평일엔 정신없이 바쁘더니만..주말인 어제와 오늘은 정말 하염없이 한가하네요..마트라...
4    남에게 관대하고 자기자신에게 엄격한 사람과남에게 엄격하고 자기자신에게 관대한 사람이...
5    주인공이 압도적인 힘으로 다 뚜까패는게 너무 좋은데 싫어하시는 분들있으신가요?뭔가 ...
6    저희 조는 파자마 스타일로 촬영했거든여근데 인형도 챙기고 왔는데하필 인형이 하츠네 ...
7                                                  ​원본
8    드디어 글로 정리하는데 성공ㅠㅠ인간은 과거의 영화를 그리워하는 것이 아니다.미래를 ...
9    infp 썸남이 고백을 2달 넘게 준비만 하고 있어요...그 사이에 뭐 카톡이나 데...
Name: posts, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] \u200b어제 저녁 8-9시쯤에 졸려서 잤는데 새벽 6시 50분에 일어났다가, 다시 자니 8시에 일어났네요 ㅎㅎ생각보다 많이 피곤했나 봐요 ㅋㅋㅋ 아직도 약간 졸려요\u200b심심ㅎ9서 FBTI 패션 성향 테스트 해봤는데\u200b\u200b이거 생각보다 맞는 거 같아요 ㅋㅋ유형별로 맞는 옷, 어울리는 옷으로 정리해둔 게 있던데 보자마자 넘 예쁘다는 생각이 들어 스크롤 내리는 중\u200b시작하기fbti.elandmall.com\u200b\u200b [SEP]',
 '[CLS] 그정도까진 아닌데 참 극단적으로 나오네요 ㅋㅋ 그래도 저번에 CCCCC 나온 것 보다는 나으려나요 [SEP]',
 '[CLS] Entj여러분, 반복된 같은 행동 잘 질려하시나요?반복해서 꾸준히 하는 게 좋다는데,\u200b저는 반복된 행동을 계속하는 걸 잘 질려해요.예를 들면 운동도 유산소 동작 3회 반복이면, 2회 같은 동작, 1회는 다른 동작으로 해야하고요.\u200b운동 종목도 수영, 달리기, 배드민턴 등 섞어서 해요 꾸준히 오래하고여..\u200b언어 공부도 예를 들어 미드로 한다고 하면 다른 종류 섞어서 들어야 꾸준히 해요...\u200b뭐든 같은 걸 계속하려면 질려해서 섞어서 해야해요.이게 entj가 단기 목표에만 강해서 그런걸까요??\u200b일할때는 어떠신가요? 저는 장기 프로젝트는 지쳐서 단기 프로젝트가 잘 맞는 것 같아요..\u200bentj가 지치지 않고 목표를 잘 달성할 수 있는여러분의 노하우를 알려주실 수 있나요?눈앞에 목표나 달성수치가 안보이면 슬럼프가 자주 오는 것 같아요! [SEP]',
 '[CLS] 평일엔 정신없이 바쁘더니만..주말인 어제와 오늘은 정말 하염없이 한가하네요..마트라 앞에 사람은 엄청 많은데.. 핸드폰은 아무 관심따위 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ진심 너무 심심해요 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ뉴스도 다 봤고.. 이거저거 하는거도 한계..어제 오늘 인터넷만 15시간은 하는모양 ㅠㅠ 뭘해야 이 심심함이 가실까요.. -_-;; [SEP]'

In [ ]:
# 라벨 추출
labels = strat_test_set['I-E'].values
labels

array([0, 0, 1, ..., 1, 0, 0])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] ​어제 저녁 8-9시쯤에 졸려서 잤는데 새벽 6시 50분에 일어났다가, 다시 자니 8시에 일어났네요 ㅎㅎ생각보다 많이 피곤했나 봐요 ㅋㅋㅋ 아직도 약간 졸려요​심심ㅎ9서 FBTI 패션 성향 테스트 해봤는데​​이거 생각보다 맞는 거 같아요 ㅋㅋ유형별로 맞는 옷, 어울리는 옷으로 정리해둔 게 있던데 보자마자 넘 예쁘다는 생각이 들어 스크롤 내리는 중​시작하기fbti.elandmall.com​​ [SEP]
['[CLS]', '어', '##제', '저', '##녁', '8', '-', '9', '##시', '##쯤', '##에', '졸', '##려', '##서', '[UNK]', '새', '##벽', '6', '##시', '50', '##분에', '일', '##어', '##났다', '##가', ',', '다시', '자', '##니', '8', '##시', '##에', '일', '##어', '##났', '##네', '##요', '[UNK]', '많이', '피', '##곤', '##했', '##나', '봐', '##요', '[UNK]', '아', '##직', '##도', '약', '##간', '[UNK]', 'F', '##B', '##TI', '패', '##션', '성', '##향', '테', '##스트', '해', '##봤', '##는데', '##이', '##거', '생', '##각', '##보다', '맞', '##는', '거', '같', '##아', '##요', '[UNK]', '맞', '##는', '옷', ',', '어', '##울', '##리는', '옷', '##으로', '정', '##리', '##해', '##둔', '게', '있던', '##데', '보', '##자', '##마', '##자', '넘', '예', '##쁘', '##다는', '생', '##각', '##이', '들어', '스', '##크', '##롤', '내', '##리는', '중', '##시', '##작', '##하기', '##f', '##bt', '##i', '.', 'elan', '##

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9546,  17730,   9663, 118738,    129,    118,    130,
        14040, 119244,  10530,   9681,  26737,  12424,    100,   9415,
       118984,    127,  14040,  10462, 110355,   9641,  12965,  58813,
        11287,    117,  25805,   9651,  25503,    129,  14040,  10530,
         9641,  12965, 118718,  77884,  48549,    100,  47058,   9946,
       118639, 119424,  16439,   9363,  48549,    100,   9519,  33077,
        12092,   9539,  18784,    100,    143,  11274,  72286,   9909,
        59095,   9434,  79544,   9866,  34994,   9960, 118991,  41850,
        10739,  41521,   9420,  66540,  80001,   9256,  11018,   8863,
         8855,  16985,  48549,    100,   9256,  11018,   9588,    117,
         9546,  78123,  26344,   9588,  11467,   9670,  12692,  14523,
       118804,   8872,  43646,  28911,   9356,  13764,  23811,  13764,
         9008,   9576, 119022,  82034,   9420,  66540,  10739,  71568,
         9477,  20308, 118882,   8996,  26344,   9694,  14040,  38709,
      

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9546,  17730,   9663, 118738,    129,    118,    130,  14040,
        119244,  10530,   9681,  26737,  12424,    100,   9415, 118984,    127,
         14040,  10462, 110355,   9641,  12965,  58813,  11287,    117,  25805,
          9651,  25503,    129,  14040,  10530,   9641,  12965, 118718,  77884,
         48549,    100,  47058,   9946, 118639, 119424,  16439,   9363,  48549,
           100,   9519,  33077,  12092,   9539,  18784,    100,    143,  11274,
         72286,   9909,  59095,   9434,  79544,   9866,  34994,   9960, 118991,
         41850,  10739,  41521,   9420,  66540,  80001,   9256,  11018,   8863,
          8855,  16985,  48549,    100,   9256,  11018,   9588,    117,   9546,
         78123,  26344,   9588,  11467,   9670,  12692,  14523, 118804,   8872,
         43646,  28911,   9356,  13764,  23811,  13764,   9008,   9576, 119022,
         82034,   9420,  66540,  10739,  71568,   9477,  20308, 118882,   8996,
         26344,   9694,  14040,  38709, 

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 모델 생성

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 모델 학습

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of    701.    Elapsed: 0:06:27.

  Average training loss: 0.63
  Training epcoh took: 0:09:02

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:20

======== Epoch 2 / 4 ========
Training...
  Batch   500  of    701.    Elapsed: 0:06:28.

  Average training loss: 0.59
  Training epcoh took: 0:09:03

Running Validation...
  Accuracy: 0.67
  Validation took: 0:00:20

======== Epoch 3 / 4 ========
Training...
  Batch   500  of    701.    Elapsed: 0:06:28.

  Average training loss: 0.54
  Training epcoh took: 0:09:04

Running Validation...
  Accuracy: 0.67
  Validation took: 0:00:20

======== Epoch 4 / 4 ========
Training...
  Batch   500  of    701.    Elapsed: 0:06:28.

  Average training loss: 0.48
  Training epcoh took: 0:09:04

Running Validation...
  Accuracy: 0.67
  Validation took: 0:00:20

Training complete!


# 테스트셋 평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    195.    Elapsed: 0:00:26.

Accuracy: 0.67
Test took: 0:00:51
